Notebook para hacer el finetuning de un transformer (distilbert pero probaremos otros). Solo estamos teniendo en cuenta el texto de la noticia.

Habrá que tener en cuenta el resto de características para ver cómo mejora. Para los estados y partido afiliado, puede ser útil relacionarlo con esto https://www.adfontesmedia.com/ de cara a la presrentación de resultados.

In [1]:
from nlp import load_dataset
dataset = load_dataset('csv', data_files={'train': 'train.csv'})
test = load_dataset('csv', data_files={'test': 'test_nolabel.csv'})

Using custom data configuration default
Using custom data configuration default


In [2]:
dataset = dataset['train'].train_test_split(test_size=0.1)

In [3]:
dataset


{'train': Dataset(features: {'id': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None), 'statement': Value(dtype='string', id=None), 'subject': Value(dtype='string', id=None), 'speaker': Value(dtype='string', id=None), 'speaker_job': Value(dtype='string', id=None), 'state_info': Value(dtype='string', id=None), 'party_affiliation': Value(dtype='string', id=None)}, num_rows: 8055),
 'test': Dataset(features: {'id': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None), 'statement': Value(dtype='string', id=None), 'subject': Value(dtype='string', id=None), 'speaker': Value(dtype='string', id=None), 'speaker_job': Value(dtype='string', id=None), 'state_info': Value(dtype='string', id=None), 'party_affiliation': Value(dtype='string', id=None)}, num_rows: 895)}

In [4]:
test

{'test': Dataset(features: {'id': Value(dtype='string', id=None), 'statement': Value(dtype='string', id=None), 'subject': Value(dtype='string', id=None), 'speaker': Value(dtype='string', id=None), 'speaker_job': Value(dtype='string', id=None), 'state_info': Value(dtype='string', id=None), 'party_affiliation': Value(dtype='string', id=None)}, num_rows: 3836)}

In [5]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch

import numpy as np
from sklearn.metrics import classification_report

In [6]:
LR = 2e-5
EPOCHS = 3
BATCH_SIZE = 16
MODEL = "distilbert-base-uncased" # use this to finetune the language model
MAX_TRAINING_EXAMPLES = -1 # set this to -1 if you want to use the whole training set

In [7]:
tokenizer = AutoTokenizer.from_pretrained(MODEL, use_fast=True)

In [8]:
train_encodings = tokenizer(dataset['train']['statement'], truncation=True, padding=True)
val_encodings = tokenizer(dataset['test']['statement'], truncation=True, padding=True)
test_encodings = tokenizer(test['test']['statement'], truncation=True, padding=True)

In [9]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

class TestDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])
    
train_dataset = MyDataset(train_encodings, dataset['train']['label'])
val_dataset = MyDataset(val_encodings, dataset['test']['label'])
test_dataset = TestDataset(test_encodings)

In [10]:
len(val_dataset)

895

In [11]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average=None)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [12]:
training_args = TrainingArguments(
    output_dir='./results',                   # output directory
    num_train_epochs=EPOCHS,                  # total number of training epochs
    per_device_train_batch_size=BATCH_SIZE,   # batch size per device during training
    per_device_eval_batch_size=BATCH_SIZE,    # batch size for evaluation
    warmup_steps=100,                         # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                        # strength of weight decay
    logging_dir='./logs',                     # directory for storing logs
    logging_steps=10,                         # when to print log
    load_best_model_at_end=True,              # load or not best model at the end
)

num_labels = len(set(dataset['train']['label']))
print(f'Num labels: {num_labels}')
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=num_labels)

Num labels: 2


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [13]:
trainer = Trainer(
    model=model,                              # the instantiated 🤗 Transformers model to be trained
    args=training_args,                       # training arguments, defined above
    train_dataset=train_dataset,              # training dataset
    eval_dataset=val_dataset,                 # evaluation dataset
    compute_metrics=compute_metrics 
)
trainer.train()

/opt/conda/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:64: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
10,0.688800
20,0.671600
30,0.645700
40,0.631700
50,0.647000
60,0.641900
70,0.660300
80,0.608200
90,0.649200
100,0.627700


TrainOutput(global_step=756, training_loss=0.5329659707331783, metrics={'train_runtime': 342.0418, 'train_samples_per_second': 2.21, 'total_flos': 4970397132748800.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 1726169088, 'init_mem_gpu_alloc_delta': 268953088, 'init_mem_cpu_peaked_delta': 206860288, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 1202065408, 'train_mem_gpu_alloc_delta': 804740096, 'train_mem_cpu_peaked_delta': 69632, 'train_mem_gpu_peaked_delta': 6544047616})

In [14]:
trainer.save_model(f"./results/best_model_{MODEL}") # save best model

In [15]:
trainer.evaluate()

{'eval_loss': 0.7217078804969788,
 'eval_accuracy': 0.6435754189944134,
 'eval_f1': array([0.49605055, 0.72428695]),
 'eval_precision': array([0.50159744, 0.71993127]),
 'eval_recall': array([0.490625  , 0.72869565]),
 'eval_runtime': 2.672,
 'eval_samples_per_second': 334.953,
 'epoch': 3.0,
 'eval_mem_cpu_alloc_delta': 135168,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_peaked_delta': 42625024}

In [16]:
PredictionOutput = trainer.predict(test_dataset)
PredictionOutput

PredictionOutput(predictions=array([[ 0.8573354 , -0.8252307 ],
       [-0.5865315 ,  0.5401295 ],
       [-1.3662791 ,  1.3618793 ],
       ...,
       [-1.1987373 ,  1.0666113 ],
       [ 0.82142735, -0.8545147 ],
       [-0.12739815, -0.09635755]], dtype=float32), label_ids=None, metrics={'test_runtime': 17.2371, 'test_samples_per_second': 222.543, 'test_mem_cpu_alloc_delta': 0, 'test_mem_gpu_alloc_delta': 0, 'test_mem_cpu_peaked_delta': 81920, 'test_mem_gpu_peaked_delta': 579115008})

In [17]:
PredictionOutput.predictions

array([[ 0.8573354 , -0.8252307 ],
       [-0.5865315 ,  0.5401295 ],
       [-1.3662791 ,  1.3618793 ],
       ...,
       [-1.1987373 ,  1.0666113 ],
       [ 0.82142735, -0.8545147 ],
       [-0.12739815, -0.09635755]], dtype=float32)

In [18]:
result = np.argmax(PredictionOutput.predictions, axis=1)

In [19]:
ids = test['test']['id']

In [20]:
import pandas as pd
df = pd.DataFrame(ids, columns=['id'])

In [21]:
df['label'] = result
df

,id,label
0,dc32e5ffa8b,0
1,aa49bb41cab,1
2,dddc8d12ac1,1
3,bcfe8f51667,1
4,eedbbaff5ab,0
...,...,...
3831,e050483b866,1
3832,6221e28aa63,0
3833,954dc0f0b5d,1
3834,2fa476b0d2f,0


In [22]:
df.to_csv('submission.csv', index=False)